In [264]:
import sys
sys.path.insert(0, "../../../")
from clases.utils import setup_environment, connect_database, sys_anio, sys_mes_ini, sys_mes_fin

setup_environment()
conn = connect_database()
anio=sys_anio()
mes_ini=sys_mes_ini()
mes_fin=sys_mes_fin()



Conexion exitosa con la Base de datos:irvin_hisminsa


In [265]:
def gestante_captadas(conn,anio,mes_ini,mes_fin):
    gestante_captadas=conn.df(f"""
  SELECT 
    provincia,
    distrito,
    --desc_ue,
    red,
    microred,
    nombre_eess,
    anio,
    mes,
    codigo_item,
    numero_documento ,
    cod_eess,
    fecha_atencion AS "fechacaptada",
   nt.valor_lab ,
   row_number() over (partition by mp.numero_documento,nt.id_establecimiento order by nt.anio,nt.fecha_atencion) as gestantes
FROM 
    maestros.nominal_trama nt  
    INNER JOIN maestros.maestro_paciente mp  ON nt.id_paciente = mp.id_paciente
    INNER JOIN maestros.eess_geresa_cusco egc ON nt.id_establecimiento = egc.id_eess
WHERE 
    nt.codigo_item LIKE ANY (ARRAY['Z349_', 'Z359_'])
    AND genero = 'F' 
    AND anio in ({anio}) 
    AND mes BETWEEN {mes_ini} AND {mes_fin} 
    AND id_tipo_documento = '1' 
    AND cat IN ('I-1', 'I-2', 'I-3', 'I-4')
    AND nt.valor_lab ='1' 
    AND egc.cod_ue !=0 --ESSALUD
 """)
    return gestante_captadas

def gestante_primer_trimestre(conn,anio,mes_ini,mes_fin):
    gestante_primer_trimestre=conn.df(f"""
SELECT DISTINCT 
    cod_eess,
    numero_documento,
    fecha_atencion,
    1 AS "gestantes_tri"
FROM 
    maestros.nominal_trama nt  
    INNER JOIN maestros.maestro_paciente mp  ON nt.id_paciente = mp.id_paciente
    INNER JOIN maestros.eess_geresa_cusco egc ON nt.id_establecimiento = egc.id_eess
WHERE 
    codigo_item IN ('Z3491', 'Z3591') 
    AND valor_lab = '1' 
    AND genero = 'F' 
    AND anio = {anio} 
    AND mes BETWEEN {mes_ini} AND {mes_fin} 
    AND id_tipo_documento = '1' 
    AND cat IN ('I-1', 'I-2', 'I-3', 'I-4')
    AND egc.cod_ue !=0 --ESSALUD
""")
    return gestante_primer_trimestre
def asignar_observaciones(st):
    if st >= 2:
        return 'REGISTRO OBSERVADO: Lab no coresponde'
    else:
        return ''

In [266]:
gestante_captadas=gestante_captadas(conn,anio,mes_ini,mes_fin)
gestante_primer_trimestre=gestante_primer_trimestre(conn,anio,mes_ini,mes_fin)

d:\www\Python-GERESA\modulos\gestante\fed\../../..\clases\bd\conexion2.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [267]:
gestante_captadas

# Agregar unir 
fed = gestante_captadas.merge( gestante_primer_trimestre, on=['numero_documento', 'cod_eess'], how='left')

trimestre_map = {
    'Z3491': 'I',
    'Z3591': 'I',
    'Z3492': 'II',
    'Z3592': 'II'
}

# Agregar una nueva columna "trimestre" basada en el mapeo
fed['trimestre'] = fed['codigo_item'].map(trimestre_map).fillna('III')



In [268]:




#fed.head()

In [269]:
#fed['gestantes_tri'] = (fed['gestantes'] == 1).astype(int)
fed['gestantes_tri'].fillna(0, inplace=True)
fed.loc[fed['gestantes'] > 1, 'gestantes_tri'] = 0  # quitar gestates observadas de numerador
fed['den'] = (fed['gestantes'] == 1).astype(int) # quitar gestates observadas de denominador
fed['obs'] = fed['gestantes'].apply(asignar_observaciones)  # observaciones 

fed = fed.drop(columns=['valor_lab', 'gestantes','codigo_item','cod_eess']) # eliminar columnas 
fed = fed.rename(columns={'gestantes_tri': 'num', 'numero_documento': 'dni'})

fed.head()

C:\Users\saga0\AppData\Local\Temp\ipykernel_25480\2997313663.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fed['gestantes_tri'].fillna(0, inplace=True)


,provincia,distrito,red,microred,nombre_eess,anio,mes,dni,fechacaptada,fecha_atencion,num,trimestre,den,obs
0,ANTA,HUAROCONDO,CUSCO NORTE,ANTA,HUAROCONDO,2024,2,40033225,2024-02-04,NaN,0.0,II,1,
1,CANCHIS,COMBAPATA,CANAS-CANCHIS-ESPINAR,COMBAPATA,COMBAPATA CANCHIS,2024,1,40199624,2024-01-20,2024-01-20,1.0,I,1,
2,CANAS,LANGUI,CANAS-CANCHIS-ESPINAR,TECHO OBRERO,LANGUI,2024,1,40477407,2024-01-25,2024-01-25,1.0,I,1,
3,ANTA,ANTA,CUSCO NORTE,ANTA,ANTA,2024,1,40493377,2024-01-10,2024-01-10,1.0,I,1,
4,CUSCO,SAN SEBASTIAN,CUSCO SUR,CUSCO,SAN SEBASTIAN,2024,1,40591689,2024-01-08,2024-01-08,1.0,I,1,


In [270]:

conn.insert_df(fed,'fed_si_01_01','materno')


DataFrame insertado en la tabla fed_si_01_01
